# The Scenario

I’ve been enrolled in the Google Data Analytics Professional Certificate since July and am currently finishing up the last course.  For the last course, they recommend working on some case studies in which we can add to our portfolios.  They gave us a couple of options to work on or we could also find and work on a case study of our own choosing.  I chose to work with the first case study they provided, How Does a Bike-Share Navigate Speedy Success?.

For this case study, I am working as a junior data analyst in the marketing team for Cyclistic, a bike-sharing company located in Chicago.  The goal of this case study is to help Cyclistic create a marketing campaign in order to maximize the number of annual memberships.  We will do so by answering the following three questions:
How do annual members and casual riders use Cyclistic bikes differently?
Why would casual riders buy Cyclistic annual memberships?
How can Cyclistic use digital media to influence casual riders to become members?
In doing so, my task is to find trends and gain insights that will help the Cyclistic executive team decide on the recommended marketing program.


# Data Sources 

The data used for analysis is from [HERE](https://divvy-tripdata.s3.amazonaws.com/index.html). The data used in the previous 12 months of Cyclistic trip data from September 2020 - August 2021.  The data is from a reliable source, Motivate International Inc under this [license](https://www.divvybikes.com/data-license-agreement) and is organized in 12 separate “.csv” files, one for each month.  Each file contains 13 attributes per observation, containing rider IDs, which location they picked up the bike to when they got off, and so on. This data will be used to find trends between casual riders and annual members. 

# Data Cleaning 

For the cleaning process, my goal is to combine all the files into one, with a consistent format throughout the file, in terms of data structures and naming conventions and so forth. I will start the process by loading all the files into a SQL database, so I can organize, sort, filter, and clean the data.  I chose to use BigQuery from the Google Cloud Platform since that is what I’ve been using throughout the course. At a quick glance of the data, I noticed that there are a bunch of nulls in various attributes that will need further inspection to see if they can be filled in as much as possible.  
After loading all the files into BigQuery, I wrote a quick query to join all the files into one.  At first, my query using UNION ALL did not run.  So upon further inspection, it seems that Station IDs started changing formats around December 2020, where instead of just integers (ex. 639), some had letters in front (ex. TA1306000029).  I decided to not include the Station IDs since we have just about the same information from the Station Names. I saved this query into another table called “all_data” to use for further data cleaning. [QUERY](https://console.cloud.google.com/bigquery?sq=198905033215:a0688ca410d3477d9d44a3b89acfe67c)

```
SELECT
  * EXCEPT ( start_station_id,
    end_station_id )
FROM
  case_study_1.trip_data_202009
UNION ALL
SELECT
  * EXCEPT ( start_station_id,
    end_station_id )
FROM
  case_study_1.trip_data_202010
UNION ALL
SELECT
  * EXCEPT ( start_station_id,
    end_station_id )
FROM
  case_study_1.trip_data_202011
UNION ALL
SELECT
  * EXCEPT ( start_station_id,
    end_station_id )
FROM
  case_study_1.trip_data_202012
UNION ALL
SELECT
  * EXCEPT ( start_station_id,
    end_station_id )
FROM
  case_study_1.trip_data_202101
UNION ALL
SELECT
  * EXCEPT ( start_station_id,
    end_station_id )
FROM
  case_study_1.trip_data_202102
UNION ALL
SELECT
  * EXCEPT ( start_station_id,
    end_station_id )
FROM
  case_study_1.trip_data_202103
UNION ALL
SELECT
  * EXCEPT ( start_station_id,
    end_station_id )
FROM
  case_study_1.trip_data_202104
UNION ALL
SELECT
  * EXCEPT ( start_station_id,
    end_station_id )
FROM
  case_study_1.trip_data_202105
UNION ALL
SELECT
  * EXCEPT ( start_station_id,
    end_station_id )
FROM
  case_study_1.trip_data_202106
UNION ALL
SELECT
  * EXCEPT ( start_station_id,
    end_station_id )
FROM
  case_study_1.trip_data_202107
UNION ALL
SELECT
  * EXCEPT ( start_station_id,
    end_station_id )
FROM
  case_study_1.trip_data_202108 AS all_data
  ```
Now that I have all the data in one combined file, what I did next was format the Station Names since I noticed they were inconsistent ([QUERY](https://console.cloud.google.com/bigquery?sq=198905033215:7fac197573c949e68a6adfdba8d2e192)). 
```
SELECT
  ride_id,
  TRIM(REPLACE(start_station_name, '(*)', '')) start_station_name,
  TRIM(REPLACE(end_station_name, '(*)', '')) end_station_name
FROM (
  SELECT
    ride_id,
    TRIM(REPLACE(start_station_name, '(Temp)', '')) start_station_name,
    TRIM(REPLACE(end_station_name, '(Temp)', '')) end_station_name
  FROM
    case_study_1.all_data)
    
```
I also changed the latitude and longitude coordinates to match with the station names ([QUERY](https://console.cloud.google.com/bigquery?sq=198905033215:e533ad9a7a794835a84cd7c8a0d1bea4)).
```
SELECT 
    DISTINCT station_name,
    ROUND(AVG(lat), 6) AS lat,
    ROUND(AVG(lng), 6) AS lng,
FROM (
    SELECT 
        DISTINCT TRIM(REPLACE(start_station_name, '(*)', '')) AS station_name, 
        AVG(lat) AS lat,
        AVG(lng) AS lng, 
    FROM (
        SELECT 
            DISTINCT TRIM(REPLACE(start_station_name, '(Temp)', '')) AS start_station_name,
            AVG(start_lat) AS lat,
            AVG(start_lng) AS lng
        FROM
            case_study_1.all_data
        GROUP BY 1) AS stat_start
    GROUP BY 
        1
    UNION DISTINCT 
    SELECT 
        DISTINCT TRIM(REPlACE(end_station_name, '(*)', '')) AS station_name,
        AVG(lat) AS lat,
        AVG(lng) AS lng
    FROM (
        SELECT 
            DISTINCT TRIM(REPLACE(end_station_name, '(Temp)', '')) AS end_station_name,
            AVG(end_lat) AS lat,
            AVG(end_lng) AS lng,
        FROM 
            case_study_1.all_data
        GROUP BY 1) AS stat_end
    GROUP BY 
        1)
GROUP BY 
    1
```
And lastly, I added two new columns for the day of the week as well as the duration of the trip in minutes ([QUERY](https://console.cloud.google.com/bigquery?sq=198905033215:2a574751461c4f35b99d21324d8749b4)).
```
SELECT
    *,
    TIMESTAMP_DIFF(ended_at, started_at, MINUTE) AS ride_length_min,
    CASE
        WHEN EXTRACT (DAYOFWEEK FROM started_at) = 1 THEN 'Sunday'
        WHEN EXTRACT (DAYOFWEEK FROM started_at) = 2 THEN 'Monday'
        WHEN EXTRACT (DAYOFWEEK FROM started_at) = 3 THEN 'Tuesday'
        WHEN EXTRACT (DAYOFWEEK FROM started_at) = 4 THEN 'Wednesday'
        WHEN EXTRACT (DAYOFWEEK FROM started_at) = 5 THEN 'Thursday'
        WHEN EXTRACT (DAYOFWEEK FROM started_at) = 6 THEN 'Friday'
        ELSE 
            'Staturday'
    END 
        AS day_of_week
FROM
    case_study_1.all_data 
```
We can use this aggregate data later in the analysis phase.  Upon further inspection, I noticed there were some negative integers under the new column for the duration of the trip.  It appears that the data was entered incorrectly for about ~1200 entries out of the 4.9 million so I deleted these entries from the dataset ([QUERY](https://console.cloud.google.com/bigquery?sq=198905033215:423e874a395148c3af24098926e1f298)).
```
DELETE 

FROM 
    case_study_1.clean_data
WHERE 
    ride_length_min < 0
```
Now that the data is processed and cleaned, I saved the new file under “clean_data” so I can use this file for analysis ([QUERY](https://console.cloud.google.com/bigquery?sq=198905033215:a4cccdaa3a7f4da7916dd795d2f54aac)).
```
WITH 
    clean_data_name AS (
        SELECT
            agg_data.ride_id,
            agg_data.rideable_type,
            agg_data.member_casual,
            agg_data.started_at,
            agg_data.ended_at,
            agg_data.day_of_week,
            agg_data.ride_length_min,
            stat_name.start_station_name,
            agg_data.start_lat,
            agg_data.start_lng,
            stat_name.end_station_name,
            agg_data.end_lat,
            agg_data.end_lng
        FROM 
            case_study_1.aggregate_data AS agg_data 
        LEFT JOIN 
            case_study_1.clean_station_name AS stat_name
        ON 
            agg_data.ride_id = stat_name.ride_id),
    clean_data_start AS(
        SELECT
            clean_agg.ride_id,
            clean_agg.rideable_type,
            clean_agg.member_casual,
            clean_agg.started_at,
            clean_agg.ended_at,
            clean_agg.day_of_week,
            clean_agg.ride_length_min,
            clean_agg.start_station_name,
            clean_lat_lng.lat AS start_lat,
            clean_lat_lng.lng AS start_lng,
            clean_agg.end_station_name,
            clean_agg.end_lat,
            clean_agg.end_lng
        FROM 
            clean_data_name AS clean_agg
        LEFT JOIN 
            case_study_1.clean_lat_lng AS clean_lat_lng
        ON 
            clean_agg.start_station_name = clean_lat_lng.station_name),
    clean_data AS(
        SELECT 
            clean_start.ride_id,
            clean_start.rideable_type,
            clean_start.member_casual,
            clean_start.started_at,
            clean_start.ended_at,
            clean_start.day_of_week,
            clean_start.ride_length_min,
            clean_start.start_station_name,
            clean_start.start_lat,
            clean_start.start_lng,
            clean_start.end_station_name,
            clean_lat_lng.lat AS end_lat,
            clean_lat_lng.lng AS end_lng
        FROM 
            clean_data_start AS clean_start
        LEFT JOIN 
            case_study_1.clean_lat_lng AS clean_lat_lng
        ON 
            clean_start.end_station_name = clean_lat_lng.station_name)
SELECT 
    *
FROM
    clean_data
```

# Analysis

Now that we have a clean and processed .csv file, I will first upload the file into R to do some initial analysis and see if there are any trends we can find. 

In [ ]:
library(tidyverse)
library(skimr)

df <- read.csv("~/R/Case_Study_1/clean_data.csv")

# Quick summary of data
skim_without_charts(df)

# General summary of avg ride duration & longest ride
summarize(df, 
          "Avg Ride Duration (Min)" = mean(ride_length_min),
          "Longest Ride (Min)" = max(ride_length_min)) 

# General avg ride duration & longest ride by day
df %>% group_by(day_of_week) %>% 
  summarize("Avg Ride Duration (Min)" = mean(ride_length_min),
            "Longest Ride (Min)" = max(ride_length_min))

# General summary of busiest & slowest day of the week
df %>% count("Busiest Day" = day_of_week) %>% slice(which.max(n)) 
df %>% count("Slowest Day" = day_of_week) %>% slice(which.min(n)) 

# Number of casuals vs members
df %>% count("status" = member_casual)

# Summary stats between casual & members - avg ride duration & longest ride
df %>% group_by("status" = member_casual) %>% 
  summarize("Avg Ride Duration (Min)" = mean(ride_length_min),
            "Longest Ride (Min)" = max(ride_length_min))

# Summary stats between casual & members by day of week - avg ride duration and longest ride
df %>% group_by("status" = member_casual, day_of_week) %>% 
  summarize("Avg Ride Duration (Min)" = mean(ride_length_min),
            "Longest Ride (Min)" = max(ride_length_min))

# Summary stats between casual & members - busiest & slowest day
df %>% group_by("status" = member_casual) %>% count("Busiest Day" = day_of_week) %>% slice(which.max(n)) 
df %>% group_by("status" = member_casual) %>% count("Slowest Day" = day_of_week) %>% slice(which.min(n))


# Summary stats between casual & members by type of bike - avg duration & longest ride
df %>% group_by("status" = member_casual, "Type of Bike" = rideable_type) %>%
  summarize("Avg Ride Duration (Min)" = mean(ride_length_min),
            "Longest Ride (Min)" = max(ride_length_min))

# Summary stats between casual & members by type of bike - busiest and slowest days
df %>% group_by("status" = member_casual, "Type of Bike" = rideable_type) %>% 
  count("Busiest Day" = day_of_week) %>% slice(which.max(n))
df %>% group_by("status" = member_casual, "Type of Bike" = rideable_type) %>% 
  count("Slowest Day" = day_of_week) %>% slice(which.min(n))

After about 50 lines of code, here are my initial findings:

* For both casual and members, the average ride time is around 23 mins, with the busiest days on the weekends.  
* However, for only casual riders, they have an average ride time of 33 mins, where as members are only around 14 mins. 
* The busiest day for casual riders was saturday and slowest day was wednesday, where as the busiest day was wednesday and slowest day was sunday for members.
* Casual members also rode with docked bikes the longest for an average of 59 mins, almost as twice as long compared to classic or electric bikes, 29 mins and 20 mins respectively.
* Member riders has and average trip of around 13 - 15 mins per type of bike.

Although we got some nice insights from the exploratory analysis from R, I will load the into [Tableau](https://public.tableau.com/app/profile/laidbackluck/viz/CaseStudy-CyclisticBike-ShareAnalysis/YearTrend) as well to see if we can gain further insights by creating visuals for analysis.

### Tableau Findings


From our first visual, we can see that riders have the most trips around mid summer to early autum, around the months of early may to late september.

As we compare the number of trips of the different types of bikes, for both members and casual riders, the most rode type of bike is the classic, followed by electric and docked respectively. 

As we look at the number of trips by day and as well as each hour, we can see that casual riders tent to have more trips on the weekends, Saturday and Sunday, and also around the afternoon hours.  
As for the member riders, they tend to have the most trips on the weekdays, Monday through Friday, and have spikes around early morning (7 & 8 AM) and the evening (5 & 6 PM), indicating members are more likely to be using the bikes to travel to and from work. 

As we visualize the initial findings of ride duration from our analysis from R, we can see that casual riders tend to ride for longer on the weekend, whereas members tend to stay roughly around the same throughout the week.  
For hours of the day however, there are some spikes throughout the day for casual riders, whereas members also stay roughly around the same.  There is a noticeably huge spike for both casual and members around 2 - 4 AM that will need further analysis. 

Finally as we look at the number of trips by station.  We can see the busiest stations tend to be around the Harbor and Gold Coast.

# Conclusions & Recommendations

From the analysis, we can see a trend of having more riders around the summer time.  We can also assume that current members tend to be riding to and from work mostly on weekday; and casual riders are more frequent on the weekends.   Also casual riders tend to ride for longer where as current members ride for a rough average of around 15 mins.  

It would be most advantageous to run a marketing campaign around summertime and on the weekends to capture the most casual riders in hopes of converting them to members.  Ideally the best locations to target would be around the harbor and Gold Coast since those tend to have the busiest stations.  We can also capitalize on the tendency of casual riders riding for longer durations by incentivizing a loyalty program, where they can get rewards based on how long they ride.